# `Trotter Costs`

(off the top of my head so probably errors, sketchy things are mostly highlighted with ??.

We want to estimate the cost of implementing time evolution of a wavefunction:

$$
|\psi(t)\rangle = e^{-i H t}|\psi(0)\rangle
$$

fault tolerantly using qualtran. The time evolution unitary can be implemented using Suzuki-Trotter methods, that is, if

$$
H = H_1 + H_2,
$$
then we can always write
$$
e^{-i H t} = \lim_{n\rightarrow\infty} \left(e^{-iH_1 t/m} e^{-iH_2 t/m}\right)^{m}
$$
This allows us to approximate the unitary as
$$
e^{-i H t} = \prod_m^{N_t} e^{-iH_1 t/N_t} e^{-iH_2 t/N_t} + \mathcal{O}(\delta t^2) =  \prod_m^{N_t} e^{-iH_1 \delta t} e^{-iH_2 \delta t} + \mathcal{O}(\delta t^2)
$$
where we have $\delta_t = t / (N_t + 1)$. More sophisticated trotter breakups lead to better accuracy, at the cost of more complicated product formula.

For the ab-initio chemistry Hamiltonian in first quantization we have

\begin{align}
H &= -\frac{1}{2} \sum_i \nabla_i^2 -\sum_{i}\sum_{J} \frac{\zeta_J}{|R_J-r_i|} + \sum_{i < j} \frac{1}{|r_i-r_j|} \\
  &= T + U + V
\end{align}

Note that the Coulomb terms are diagonal in the position basis while the kinetic term is diagonal in the momentum basis. Thus we can employ a QFT: 
$$
|\psi(t)\rangle \approx \mathrm{QFT} e^{-i\delta t T} \mathrm{QFT}^{\dagger} e^{-i\delta t U}  e^{-i \delta t V} |\psi(0)\rangle
$$
so that all the terms can be implemented via a gate which implements something of the form $e^{-i \delta t \phi({r_i}) }$ via a phasing gate.

Specifically, we are concerned with implementing time evolution in the first quantized representation using a grid based approach. Our wavefunction will be represented on a real space grid 

$$
|\psi\rangle = \sum_{r_1\cdots r_\eta} c(r_1, \cdots, r_\eta) |r_1\cdots r_\eta\rangle
$$

and $r$ lives on a grid of size $N = (2 N_g + 1)^3$ if $N_g$ is the number of grid points in each spatial dimension. Thus we have $\eta$ registers of size $\log N$.

According to Jones et al., the main steps to implement are (for the electron-electron interaction $V$, but the other terms are similar)

\begin{align}
&\sum_{r_1\cdots r_\eta} c(r_1, \cdots, r_\eta) |r_1\cdots r_\eta\rangle \\
  &\rightarrow  \sum_{r_1\cdots r_\eta} c(r_1, \cdots, r_\eta) |r_1\cdots r_\eta\rangle|V(r_1\cdots r_\eta)\rangle \hspace{10em} \text{Compute pairwise potential in ancilla registers} \\
  &\rightarrow \sum_{r_1\cdots r_\eta} e^{-i V(r_1\cdots r_\eta)} c(r_1, \cdots, r_\eta) |r_1\cdots r_\eta\rangle|V(r_1\cdots r_\eta)\rangle \hspace{5.5em} \text{Phase the state with computed potential} \\ 
  &\rightarrow \sum_{r_1\cdots r_\eta} e^{-i V(r_1\cdots r_\eta)} c(r_1, \cdots, r_\eta) |r_1\cdots r_\eta\rangle|0\cdots0\rangle \hspace{7.8em} \text{Uncompute potential in ancilla register} \\ 
\end{align}
in the above the ancilla register storing the value of the potential is of size.

To compute the potential we need to compute $\frac{1}{r_{ij}}$ which can be done in two steps: 1) compute $|r_{ij}^2\rangle = |(x_i - x_j)^2 + (y_i-y_j)^2 + (z_i - z_j)^2\rangle$ into a register of size $2 \log N$ (??, assuming any grid spacing prefactors are accounted for elsewhere), and 2) compute the inverse square root of the number in this register ($r_{ij}$).

To compute $\frac{1}{r_{ij}}$ we write solve $x^{-2} = r_{ij}^2$ which has solution $x^* = \frac{1}{r_{ij}}$. This can be solved iteratively using the Newton-Raphson method: 

\begin{align}
a_{n+1} &= a_{n} - \frac{a_n^3(1-r_{ij}^2 a_n^2)}{-2 a_n^2} \\
        &= \frac{1}{2}a_n\left(3-a_n^2 r_{ij}^2\right).
\end{align}
In the fusion paper this method is improved upon by using a hybrid approach based upon QROM function interpolation in conjunction with this Newton-Raphson iteration. A further optimization is introduced to include the scaling factors necessary for the potential directly in the Newton-Raphson step (i.e. all the timestep and factor of two factors.)